# `mult` processor demo

In [ ]:
USE_PRIVATE_DISTRO = True

%tensorflow_version 2.x
if USE_PRIVATE_DISTRO:
    print("[INFO] Using private distro. Be careful.")
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -qU /content/drive/MyDrive/SMC\ 10/DDSP-10/dist/ddsp-1.2.0.tar.gz
else:
    !pip install -qU ddsp

# Ignore a bunch of deprecation warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf

import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

import ddsp
import ddsp.training
from ddsp.colab.colab_utils import play, specplot

In [ ]:
SAMPLE_RATE = 16000
DURATION = 4

N_SAMPLES = SAMPLE_RATE * DURATION

### Define processors and synths

In [ ]:
carrier = ddsp.synths.Harmonic(n_samples=N_SAMPLES, 
                            sample_rate=SAMPLE_RATE, 
                            name='carrier')
lfo = ddsp.synths.Harmonic(n_samples=N_SAMPLES, 
                            sample_rate=SAMPLE_RATE, 
                            name='lfo')
mult = ddsp.processors.Mult(name='mult')
add = ddsp.processors.Add(name='add')

In [ ]:
n_harmonics = 1

# Create a sinusoid
carrier_gain = np.ones(1)[np.newaxis, :, np.newaxis]
carrier_harmonic_amps = np.ones((1, n_harmonics))[np.newaxis, :, :]
carrier_f0_hz = 440.0 * np.ones(1)[np.newaxis, :, np.newaxis]

# Create an LFO
lfo_gain = np.ones(1)[np.newaxis, : , np.newaxis]
lfo_harmonic_amps = np.ones((1,n_harmonics))[np.newaxis, :, :]
lfo_f0_hz = 220.0 * np.ones(1)[np.newaxis, :, np.newaxis]

inputs = {
    'carrier_gain': carrier_gain,
    'carrier_harmonic_amps': carrier_harmonic_amps,
    'carrier_f0_hz': carrier_f0_hz,
    'lfo_gain': lfo_gain,
    'lfo_harmonic_amps': lfo_harmonic_amps,
    'lfo_f0_hz': lfo_f0_hz,
}
inputs = {k: v.astype(np.float32) for k, v in inputs.items()}

### Define DAG

In [ ]:
dag = [
  (carrier, ['carrier_gain', 'carrier_harmonic_amps', 'carrier_f0_hz']),
  (lfo, ['lfo_gain', 'lfo_harmonic_amps', 'lfo_f0_hz']),
  (mult, ['carrier/signal', 'lfo/signal']),
  (add, ['carrier/signal', 'mult/signal']),
]
processor_group = ddsp.processors.ProcessorGroup(dag=dag)
controls = processor_group.get_controls(inputs)

### Ring modulation

In [ ]:
audio = controls['mult']['signal'] / np.max(np.abs(controls['mult']['signal']))

play(audio, sample_rate=SAMPLE_RATE)
specplot(audio)

f, ax = plt.subplots(1, 2, figsize=(10.5, 3), sharey=True)
ax[0].plot(controls['carrier']['signal'][0,:256])
ax[0].plot(controls['lfo']['signal'][0,:256])
ax[0].set_ylabel('Amplitude')
ax[0].set_xlabel('Samples')
ax[0].legend(['Carrier', 'LFO'])
ax[1].plot(audio[0,:256])
ax[1].legend(['Output (Ring mod.)'])
_ = ax[1].set_xlabel('Samples')

### Amplitude modulation

In [ ]:
audio = controls['add']['signal'] / np.max(np.abs(controls['add']['signal']))

play(audio, sample_rate=SAMPLE_RATE)
specplot(audio)

f, ax = plt.subplots(1, 2, figsize=(10.5, 3), sharey=True)
ax[0].plot(controls['carrier']['signal'][0,:256])
ax[0].plot(controls['lfo']['signal'][0,:256])
ax[0].set_ylabel('Amplitude')
ax[0].set_xlabel('Samples')
ax[0].legend(['Carrier', 'LFO'])
ax[1].plot(audio[0,:256])
ax[1].legend(['Output (Ring mod.)'])
_ = ax[1].set_xlabel('Samples')